In [1]:
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [2]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=109, max_dist=8.090562782738231e-05, ..., log_cap=-0.42683289627564536)

We can also access the scaling matrices and the final scaled state:

In [3]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[ 1.65786657+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.40308102+0.39223556j  1.00312688+0.j          0.        +0.j        ]
 [ 0.00758933+0.11396695j -0.49548987-0.14238749j  0.55880939+0.j        ]] 

[[ 0.61763252+0.j          0.        +0.j          0.        +0.j        ]
 [-0.1727828 -0.68997742j  1.70829184+0.j          0.        +0.j        ]
 [-2.59969632+1.01563751j  0.17507415-4.0453014j   1.56045881+0.j        ]] 

[[ 1.9724628 +0.j          0.        +0.j          0.        +0.j        ]
 [ 0.00869827+0.19941352j  0.44007128+0.j          0.        +0.j        ]
 [-0.51598731-0.22684528j -0.17369052-0.27674274j  0.39078995+0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [4]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=100)

Result(success=False, iterations=100, max_dist=0.5934653055554957, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [5]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (np.float64(0.5), np.float64(0.5))
  1: (np.float64(0.5), np.float64(0.5))
  2: (np.float64(0.5), np.float64(0.5))
  3: (np.float64(0.5), np.float64(0.5))
#000: max_dist = 0.35355339 @ sys = 1
#001: max_dist = 0.47140452 @ sys = 0
#002: max_dist = 0.56568542 @ sys = 2
#003: max_dist = 0.62853936 @ sys = 3
#004: max_dist = 0.58232323 @ sys = 1
#005: max_dist = 0.59305730 @ sys = 0
#006: max_dist = 0.59545834 @ sys = 2
#007: max_dist = 0.59262283 @ sys = 3
#008: max_dist = 0.59353004 @ sys = 1
#009: max_dist = 0.59357133 @ sys = 0
#010: max_dist = 0.59340661 @ sys = 2
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [6]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=35, max_dist=5.735258641525422e-07, ..., log_cap=-0.2521460932027963)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [7]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[ 0.38009+0.j       0.00741-0.00678j  0.01028+0.01862j]
 [ 0.00741+0.00678j  0.30533+0.j      -0.0029 +0.00376j]
 [ 0.01028-0.01862j -0.0029 -0.00376j  0.31457+0.j     ]] 

[[ 0.4+0.j  0. +0.j -0. +0.j]
 [ 0. -0.j  0.3+0.j  0. -0.j]
 [-0. -0.j  0. +0.j  0.3+0.j]] 

[[0.7+0.j 0. -0.j 0. -0.j]
 [0. +0.j 0.2+0.j 0. -0.j]
 [0. -0.j 0. +0.j 0.1+0.j]]
